In [31]:
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm
import pgpy
import os
import glob
from secrets import token_hex

In [32]:
def key_generation(user_name,passphrase,salt):
    key = pgpy.PGPKey.new(PubKeyAlgorithm.RSAEncryptOrSign, 2048)
    uid = pgpy.PGPUID.new(user_name)

    key.add_uid(uid, usage={KeyFlags.Sign, KeyFlags.EncryptCommunications, KeyFlags.EncryptStorage},
            hashes=[HashAlgorithm.SHA256, HashAlgorithm.SHA384, HashAlgorithm.SHA512, HashAlgorithm.SHA224],
            ciphers=[SymmetricKeyAlgorithm.AES256, SymmetricKeyAlgorithm.AES192, SymmetricKeyAlgorithm.AES128],
            compression=[CompressionAlgorithm.ZLIB, CompressionAlgorithm.BZ2, CompressionAlgorithm.ZIP, CompressionAlgorithm.Uncompressed])
    key.protect(passphrase+salt, SymmetricKeyAlgorithm.AES256, HashAlgorithm.SHA256)
    return key

In [33]:
# message is in string given by User
# pub_key is fetched from Database i.e Receivers public Key
def encryption(pub_key,message):
    pub_key_enc_pgpy, _ = pgpy.PGPKey.from_blob(pub_key)
    text_message = pgpy.PGPMessage.new(message)
    encrypted_message = pub_key_enc_pgpy.encrypt(text_message)
    return encrypted_message

In [34]:
# sec_key is Private key stored on local device
# enc_message is encrypted message
def decryption(sec_key,enc_message,passphrase,salt):
    temp_pgpy = pgpy.PGPKey()
    temp_pgpy.parse(sec_key)
    try:
        with temp_pgpy.unlock(passphrase+salt):
            decrypted_message = temp_pgpy.decrypt(enc_message)
            return decrypted_message.message
    except:
        print("Wrong passphrase")

In [35]:
# sec_key is key stored at local machine i.e private key
# message is string
def sign(sec_key,message,passphrase,salt):
    temp_pgpy = pgpy.PGPKey()
    temp_pgpy.parse(sec_key)
    try:
        with temp_pgpy.unlock(passphrase+salt):
            message = pgpy.PGPMessage.new(message)
            signature = temp_pgpy.sign(message)
            return signature
    except:
        print("Wrong passphrase")

In [36]:
# pub_key is key fetched from DB
# message is signed string message
def verify(pub_key,message,signature):
    pub_key, _ = pgpy.PGPKey.from_blob(pub_key)
    temp_pgpy = pgpy.PGPSignature()
    temp_pgpy.parse(signature)
    if pub_key.verify(message,temp_pgpy):
        return True
    else:
        return False


In [37]:
def get_salt():
    return str(token_hex(16))

In [8]:
# INFO=======================================
# store the OUTPUT of key_generation function on local machine as private key
# Get public key using key.pubkey method and store it on DB

In [38]:
# Sender Side
salt1 = get_salt()
sender_key = key_generation('sender',"SecretPassphrase1",salt1)
# print(sender_key.pubkey)

In [39]:
#Receiver Side
salt2=get_salt()
receiver_key = key_generation('receiver',"SecretPassphrase2",salt2)
# print(receiver_key.pubkey)

In [40]:
sender_sec_str = str(sender_key) # sec_str is key from local machine
sender_pub_str = str(sender_key.pubkey) # pub_str is key from DB

In [41]:
receiver_sec_str = str(receiver_key) # sec_str is key from local machine
receiver_pub_str = str(receiver_key.pubkey) # pub_str is key from DB

In [42]:
#Sender Side
enc = encryption(receiver_pub_str,"This is Time Pass Message!")
print(enc)
print(type(enc))
print("encrypted? " + str(enc.is_encrypted))
print("Signed? " + str(enc.is_signed))

-----BEGIN PGP MESSAGE-----

wcBMA1n4AwvTzPYlAQgAg/qJ44Eh47VaizhXoFEDG0aKw2Gg0KBrArYPIkSrayCE
GGqOD0yDTJKHPQSjqdKdRa7hjAedVQZEyR5wTwr+WFB1MfyHjkMAUwxWfuAdZYzC
Cm07Up/MZ7YdG7TJbhd9iMf/nOsht6kazr+VMiRmCoa2wihAE7/guhZT4Z9zGDb/
NhaElMpKg067Jjck71UjCLNViFGDpZ7d37IoEG+L0/WSEUwYUpzIrmF+1DbP6pmS
kxqONA3vxRq1/5NTMAiNgLOJxjwReg5A6VFyw8KFEPBhYhwhrOlqRpYHoB9XbPwl
E5VYjLC0rM1i4N7IYuwKIeeEW6fm+sFrfNNuGgvol9JOAWDVn/GoSWMaR2IfGC3+
D2qBHZJerl4DCARxWD1GUlp1bTkV2sWs1Dcr0NVy/i/3iZ47oXTpUNQ5SoKEFEqt
o7haodTFn717BhLCYvHC
=ekZn
-----END PGP MESSAGE-----

<class 'pgpy.pgp.PGPMessage'>
encrypted? True
Signed? False


In [43]:
#Sender Side
signature = sign(sender_sec_str,str(enc),"SecretPassphrase1",salt1)
print(signature)
print(type(signature))

-----BEGIN PGP SIGNATURE-----

wsBcBAABCAAGBQJepWEfAAoJEENvgUtxXTwx5VQIALto3eiBgLFz8CGgtZ8DDjgC
9h1Q6vEMXQy4/YlKlqTWTKPGbDa179avZAm16n0m7PKg6HxUu9CPEB3NZ4pnmp/L
nzYM0XvIybxWjVjEeCICADlsBfF9Qtoain1FwFxI/TdXXnxsDMJjvha2LMpjo1AA
/hy+MsbrFg4/2Ui+Q8ZQoJ1w+I+GgBXzvYJ3lSt1e9fAc+5wpQ7D6Xr4y0siSt5M
zeS2uN5RG7SSFLAI/egBOjXFv8FBCAcL2j2SYDLboH5G/0mK4++3okYS9IoG2KR2
3qqhXXHcruBS66U8PBQ/NT56wQGk3Ro9vVrJD4iliIleGgi2rpZu83cOcepNLXw=
=mZE9
-----END PGP SIGNATURE-----

<class 'pgpy.pgp.PGPSignature'>


In [44]:
#Receiver Side
print(verify(sender_pub_str,str(enc),str(signature)))

True


In [45]:
#Receiver Side
print(decryption(receiver_sec_str,enc,"SecretPassphrase2",salt2))

This is Time Pass Message!
